In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

 # xopt CNSGA example

In [2]:
from xopt.cnsga import cnsga
import json
import numpy as np
import matplotlib.pyplot as plt

In [3]:
NAME = 'TNK'
BOUND_LOW, BOUND_UP = [0.0, 0.0], [3.14159, 3.14159]  


X_RANGE = [0, 1.4]
Y_RANGE = [0, 1.4]

# Pure number version
def TNK(individual):    
    x1=individual[0]
    x2=individual[1]
    objectives =  (x1, x2)
    constraints = (x1**2+x2**2-1.0 - 0.1*np.cos(16*np.arctan2(x1, x2)), 0.5-(x1-0.5)**2-(x2-0.5)**2 )
    return objectives, constraints

# labeled version
def evaluate_TNK(inputs):
    
    info = {'some':'info', 'about':['the', 'run']}
    ind = [inputs['x1'], inputs['x2']]
    objectives, constraints = TNK(ind)    
    outputs = {'y1':objectives[0], 'y2':objectives[1], 'c1':constraints[0], 'c2':constraints[1]}
    
    return outputs

VOCS = {
    'name':'TNK_test',
    
    'variables': {
        'x1':[0, 3.14159],
        'x2':[0, 3.14159]
    },
    'objectives':{
        'y1':'MINIMIZE',
        'y2':'MINIMIZE'
        
    },
    'constraints':{
        'c1': ['GREATER_THAN', 0],
        'c2': ['GREATER_THAN', 0]
        
    },
    'constants':{'a':'dummy_constant'},
    'linked_variables':{'x9':'x1'}
    
    
}


In [4]:
# Pick a parallel method


#from concurrent.futures import ProcessPoolExecutor as PoolExecutor
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [16]:
#pop = cnsga(executor, toolbox=TOOLBOX, max_generations = 40, population_size=64) 
pop_data = cnsga(executor, vocs=VOCS, evaluate_f=evaluate_TNK,
            max_generations = 20, population_size=64, output_path=None,
            verbose=True) 

Creating toolbox from vocs.
Warning in cnsga_toolbox: Redefining creator.Individual
Created toolbox with 2 variables, 2 constraints, and 2 objectives.
    Using selection algorithm: nsga2
____________________________________________________
64 fitness calculations for initial generation
................................................................done.
Submitting first batch of children
.................................................................
__________________________________________________________
Generation 0 completed in 0.00091 minutes
................................................................
__________________________________________________________
Generation 1 completed in 0.00167 minutes
................................................................
__________________________________________________________
Generation 2 completed in 0.00181 minutes
................................................................
___________________________________________

In [ ]:
pop_data2 = cnsga(executor, vocs=VOCS, population=pop_data, evaluate_f=evaluate_TNK,
            max_generations = 40, population_size=64, output_path=None,
            verbose=True) 

In [ ]:
pop_init(VOCS, pop_data)

In [10]:
for i in range(10):
    print('.', end='')

..........

In [ ]:
cnsga(executor, vocs=VOCS, population=pop['variables'], evaluate_f=evaluate_TNK,
            max_generations = 40, population_size=64,
            verbose=True) 

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

front = np.array([ind.fitness.values for ind in pop])
ax.scatter(front[:,0], front[:,1], color='blue')
ax.set_xlim(X_RANGE)
ax.set_ylim(Y_RANGE)
ax.set_aspect('auto')
ax.set_title(NAME)

In [ ]:
import sys
assert sys.version_info >= (2, 5)

In [ ]:
import sys
assert sys.version_info >= (3, 6)